### Data Structure  
This data is about a subscription-based digital product offering for financial advisory that includes newsletters, webinars, and investment recommendations. The offering has a couple of varieties, annual subscription, and digital subscription. The product also provides daytime support for customers to reach out to a care team that can help them with any product-related questions and signup/cancellation-related queries.  
1. product_info encompasses product names, corresponding pricing details, and the duration of each subscription plan.    
2. customer_info includes customer demographics like age and gender.   
3. customer_product details customer sign-up and cancellation dates at the product level, if they have not yet ended their subscription cancel date is null.    
4. customer_cases captures records of customer contacts with support, including timestamps, contact methods (like phone or email), and the reason for each contact.  











####Product Info

In [0]:
%sql
select * from product_info;

product_id,name,price,billing_cycle
prd_1,annual_subscription,1200,12
prd_2,monthly_subscription,125,1


In [0]:
%sql

describe table product_info;

col_name,data_type,comment
product_id,string,null
name,string,null
price,int,null
billing_cycle,int,null


####Customer Info

In [0]:
%sql
select
  *
from
  customer_info
limit
  5;

_c0,customer_id,age,gender,age_bracket
1,C2448,76,female,70-79
2,C2449,61,male,60-69
3,C2450,58,female,50-59
4,C2451,62,female,60-69
5,C2452,71,male,70-79


In [0]:
%sql
describe table customer_info;

col_name,data_type,comment
_c0,int,null
customer_id,string,null
age,int,null
gender,string,null
age_bracket,string,null


In [0]:
%sql
select distinct(gender)
from customer_info;

gender
female
male


####Customer Product

In [0]:
%sql
select * from customer_product
limit 5;

signup_id,customer_id,product,signup_date_time,cancel_date_time
1,C2448,prd_1,2017-01-01T10:35:00.000+0000,null
2,C2449,prd_1,2017-01-01T11:39:00.000+0000,2021-09-05T10:00:00.000+0000
3,C2450,prd_1,2017-01-01T11:42:00.000+0000,2019-01-13T16:24:00.000+0000
4,C2451,prd_2,2017-01-01T13:32:00.000+0000,null
5,C2452,prd_1,2017-01-01T13:57:00.000+0000,2021-06-28T18:06:00.000+0000


In [0]:
%sql
describe table customer_product;

col_name,data_type,comment
signup_id,int,null
customer_id,string,null
product,string,null
signup_date_time,timestamp,null
cancel_date_time,timestamp,null


In [0]:
%sql
select distinct(product) from customer_product;

product
prd_1
prd_2


####Customer Cases

In [0]:
%sql
select * from customer_cases
limit 5;

_c0,case_id,date_time,customer_id,channel,reason
1,CC101,2017-01-01T10:32:03.000+0000,C2448,phone,signup
2,CC102,2017-01-01T11:35:47.000+0000,C2449,phone,signup
3,CC103,2017-01-01T11:37:09.000+0000,C2450,phone,signup
4,CC104,2017-01-01T13:28:14.000+0000,C2451,phone,signup
5,CC105,2017-01-01T13:52:22.000+0000,C2452,phone,signup


In [0]:
%sql
describe table customer_cases;

col_name,data_type,comment
_c0,int,null
case_id,string,null
date_time,timestamp,null
customer_id,string,null
channel,string,null
reason,string,null


In [0]:
%sql
select distinct(channel) from customer_cases;

channel
phone
email


In [0]:
%sql
select distinct(reason) from customer_cases;

reason
signup
support


### Exploratory Data Analysis

In [0]:
%sql
-- Product distribution subscribers
select
  name as Product_Name,
  count(*) as count,
  round(count(*) / sum(count(signup_id))over(), 2) as percentage
from
  customer_product
  left join product_info on customer_product.product = product_info.product_id
group by
  name;

Product_Name,count,percentage
annual_subscription,325649,0.64
monthly_subscription,183283,0.36


Around 65% customers took annual subscription and 35% took monthly

In [0]:
%sql
-- Gender distribution of subscribers
select
  gender,
  count(*) as count,
  round(count(*) / sum(count(signup_id))over(), 2) as percentage
from
  customer_product
  left join customer_info on customer_product.customer_id = customer_info.customer_id
group by
  gender;

gender,count,percentage
female,199002,0.39
male,309930,0.61


39% are female customers and 61% are male

In [0]:
%sql
-- gender and product distribution
select
  gender,
  annual,
  monthly
from
  (
    SELECT
      customer_info.gender,
      customer_product.product
    from
      customer_product
      left join customer_info on customer_product.customer_id = customer_info.customer_id
  ) as cp pivot (
    count(*) as customers for product in ('prd_1' as annual, 'prd_2' as monthly)
  )


gender,annual,monthly
female,126420,72582
male,199229,110701


In [0]:
%sql
select max(age) max_age, min(age) min_age, round(avg(age),0) avg_age
from customer_info;

max_age,min_age,avg_age
78,21,58.0


In [0]:
%sql
select age_bracket, count(*) as count, round(count(*) / sum(count(customer_id))over(), 2) as percentage
from customer_info
group by age_bracket
order by age_bracket;

age_bracket,count,percentage
20-29,1001,0.0
30-39,9138,0.02
40-49,60209,0.12
50-59,195379,0.38
60-69,202512,0.4
70-79,40693,0.08


80% Customers are from age 50-70

In [0]:
%sql
select channel, count(*) as count,  round(count(*) / sum(count(case_id))over(), 2) as percentage
from customer_cases
group by channel;

channel,count,percentage
phone,286840,0.87
email,43672,0.13


In [0]:
%sql
select reason, count(*) as count, round(count(*) / sum(count(case_id))over(), 2) as percentage
from customer_cases
group by reason;

reason,count,percentage
signup,129527,0.39
support,200985,0.61


In [0]:
%sql
select
  age_bracket,
  phone,
  email
from
  (
    select
      age_bracket,
      channel
    from
      customer_cases
      left join customer_info on customer_cases.customer_id = customer_info.customer_id
  ) as chan
  pivot (
    count(*) as customers for channel in ('phone' as phone, 'email' as email)
  )
  order by age_bracket;

age_bracket,phone,email
20-29,1279,97
30-39,9953,782
40-49,43323,5218
50-59,102150,16657
60-69,100357,17435
70-79,29778,3483


In [0]:
%sql
select
  age_bracket,
  signup,
  support
from
  (
    select
      age_bracket,
      reason
    from
      customer_cases
      left join customer_info on customer_cases.customer_id = customer_info.customer_id
  ) as age pivot (
    count(*) as customers for reason in ('signup' as signup, 'support' as support)
  )
  order by age_bracket;


age_bracket,signup,support
20-29,964,412
30-39,7020,3715
40-49,24265,24276
50-59,42196,76611
60-69,38147,79645
70-79,16935,16326


**channel wise:** it is clear that 87% people tend to use phone over email across all age brackets.  
**reason wise:** as age increase the ratio of support request kept increasing 

In [0]:
%sql
select
  channel,
  signup,
  support
from
  (
    select
      channel,
      reason
    from
      customer_cases
  ) as ch pivot (
    count(*) as customers for reason in ('signup' as signup, 'support' as support)
  )
  

channel,signup,support
phone,129527,157313
email,null,43672


The absence of email signups is a potential area for improvement. Introducing this option could streamline the process for both customers and staff, saving time and effort.

### Churn Analysis

In [0]:
%sql
WITH GroupedYear AS (
  select
    *,
    (year(signup_date_time) + 1) AS churn_year
  FROM
    customer_product
),
CustomersAtYearStart as (
  SELECT
    churn_year,
    count(customer_id) as custs
  FROM
    GroupedYear
  where
    year(cancel_date_time) >= churn_year
    or year(cancel_date_time) is null
  GROUP BY
    churn_year
),
cancelledCustomers AS (
  select
    year(cancel_date_time) as churn_year,
    count(cancel_date_time) as cancelled
  FROM
    customer_product
  where
    cancel_date_time IS NOT NULL
  group by
    year(cancel_date_time)
)
SELECT
  G.churn_year,
  round((C.cancelled / G.custs) * 100, 2) as churn_rate
from
  CustomersAtYearStart G
  inner join cancelledCustomers C on C.churn_year = G.churn_year
order by
  G.churn_year

churn_year,churn_rate
2018,22.14
2019,29.01
2020,31.6
2021,41.47


Churn rates have steadily increased over the years, with particularly sharp spikes observed in 2019 and 2021.

In [0]:
%sql
SELECT
  YEAR(date_time) AS year,
  COUNT(case_id) AS cases,
  (
    count(case_id) - lag(count(case_id), 1) OVER (
      ORDER BY
        year(date_time)
    )
  ) / lag(count(case_id), 1) OVER (
    ORDER BY
      year(date_time)
  ) * 100 as yoy_change
FROM
  customer_cases
WHERE
  year(date_time) NOT IN (2022)
GROUP BY
  YEAR(date_time)
ORDER BY
  year;

year,cases,yoy_change
2017,17823,null
2018,34193,91.84761263535881
2019,61566,80.0543970988214
2020,81390,32.19959068316928
2021,135539,66.53028627595528


Overall, the data suggests a consistent increase in the number of cases, with a significant acceleration in the early years and a less year-over-year change from 2020. But, Year 2021 has highest contribution in the number of cases, indicating a possible glitch in the services or increase in signups.

In [0]:
%sql
select
year,
signup,
support 
from(
  select
  year(date_time) as year,
  reason,
  round(count(case_id)*100/sum(count(case_id)) over(PARTITION BY reason),2) as percentage
from
  customer_cases
group by
  year(date_time),
  reason) as pv pivot(
    max(percentage) for reason in ('signup' as signup, 'support' as support)
  )
order by year;

year,signup,support
2017,8.05,3.68
2018,13.06,8.59
2019,23.19,15.68
2020,26.66,23.32
2021,29.04,48.72
2022,null,0.0


The majority of support calls were concentrated in 2021, suggesting a potential widespread issue that affected a significant number of customers. To maintain a healthy churn rate, it's essential to address this issue and also focus on rate of increase in signups, which have been declining since 2019

In [0]:
%sql
WITH GroupedYear AS (
  select
    gender,
    customer_info.customer_id as customer_id,
    (year(signup_date_time) + 1) AS churn_year,
    cancel_date_time
  FROM
    customer_product
    left join customer_info on customer_info.customer_id = customer_product.customer_id
),
CustomersAtYearStart as (
  SELECT
    churn_year,
    gender,
    count(customer_id) as custs
  FROM
    GroupedYear
  where
    year(cancel_date_time) >= churn_year
    or year(cancel_date_time) is null
  GROUP BY
    churn_year,
    gender
  order by
    churn_year,
    gender
),
cancelledCustomers AS (
  select
    gender,
    year(cancel_date_time) as churn_year,
    count(cancel_date_time) as cancelled
  FROM
    customer_product
    left join customer_info on customer_info.customer_id = customer_product.customer_id
  where
    cancel_date_time IS NOT NULL
  group by
    year(cancel_date_time),
    gender
  order by
    churn_year,
    gender
)
select
  churn_year,
  female,
  male
from(
    SELECT
      G.churn_year as churn_year,
      round((C.cancelled / G.custs) * 100, 2) as churn,
      G.gender as gender
    from
      CustomersAtYearStart G
      inner join cancelledCustomers C on C.churn_year = G.churn_year
      and C.gender = G.gender
  ) as ch pivot (
    max(churn) as churn for gender in ('male' as male, 'female' as female)
  )

churn_year,female,male
2018,21.86,22.33
2019,29.63,28.6
2020,31.55,31.63
2021,41.66,41.35


Gender does not appear to significantly influence churn rates. Both male and female customers exhibit similar patterns of churn

In [0]:
%sql
WITH GroupedYear AS (
  select
    product,
    customer_id,
    (year(signup_date_time) + 1) AS churn_year,
    cancel_date_time
  FROM
    customer_product

),
CustomersAtYearStart as (
  SELECT
    churn_year,
    product,
    count(customer_id) as custs
  FROM
    GroupedYear
  where
    year(cancel_date_time) >= churn_year
    or year(cancel_date_time) is null
  GROUP BY
    churn_year,
    product
  order by
    churn_year,
    product
),
cancelledCustomers AS (
  select
    product,
    year(cancel_date_time) as churn_year,
    count(cancel_date_time) as cancelled
  FROM
    customer_product
   
  where
    cancel_date_time IS NOT NULL
  group by
    year(cancel_date_time),
    product
  order by
    churn_year,
    product
)
select
  churn_year,
  prd_1,
  prd_2
from(
    SELECT
      G.churn_year as churn_year,
      round((C.cancelled / G.custs) * 100, 2) as churn,
      G.product as product
    from
      CustomersAtYearStart G
      inner join cancelledCustomers C on C.churn_year = G.churn_year
      and C.product = G.product
  ) as ch pivot (
    max(churn) for product in ('prd_1' as prd_1, 'prd_2' as prd_2)
  )

churn_year,prd_1,prd_2
2018,20.04,32.41
2019,25.59,42.54
2020,26.68,47.85
2021,36.59,49.88


Churn rates exhibit distinct patterns for annual and monthly subscription models, potentially influenced by variations in cancellation rates or customer acquisition challenges

In [0]:
%sql
WITH GroupedYear AS (
  select
    product,
    customer_id,
    (year(signup_date_time) + 1) AS churn_year,
    cancel_date_time
  FROM
    customer_product
),
CustomersAtYearStart as (
  SELECT
    churn_year,
    product,
    count(customer_id) as custs
  FROM
    GroupedYear
  where
    year(cancel_date_time) >= churn_year
    or year(cancel_date_time) is null
  GROUP BY
    churn_year,
    product
  order by
    churn_year,
    product
),
cancelledCustomers AS (
  select
    product,
    year(cancel_date_time) as churn_year,
    count(cancel_date_time) as cancelled
  FROM
    customer_product
  where
    cancel_date_time IS NOT NULL
  group by
    year(cancel_date_time),
    product
  order by
    churn_year,
    product
)
SELECT
  G.churn_year as churn_year,
  G.product as product,
  (C.cancelled - lag(C.cancelled, 1) over(
    ORDER BY
      G.product,
      G.churn_year
  )) / lag(C.cancelled, 1) over(
    ORDER BY
      G.product,
      G.churn_year) * 100
   as yoy_cancelled,
   (G.custs - lag(G.custs, 1) over(
    ORDER BY
      G.product,
      G.churn_year
  )) / lag(G.custs, 1) over(
    ORDER BY
      G.product,
      G.churn_year) * 100
   as yoy_customers,
  
  round((C.cancelled / G.custs) * 100, 2) as churn
from
  CustomersAtYearStart G
  inner join cancelledCustomers C on C.churn_year = G.churn_year
  and C.product = G.product
ORDER BY
  product,
  churn_year

churn_year,product,yoy_cancelled,yoy_customers,churn
2018,prd_1,null,null,20.04
2019,prd_1,130.40694071472836,80.435412441538,25.59
2020,prd_1,88.92773892773893,81.1652712467026,26.68
2021,prd_1,48.15166326579035,8.031248812975601,36.59
2018,prd_2,-94.85586162716207,-94.1914160474438,32.41
2019,prd_2,191.84308841843088,122.31638418079096,42.54
2020,prd_2,143.80200554725837,116.73624977309855,47.85
2021,prd_2,116.1372188675943,107.36599664991624,49.88




**Product 1:** In 2019 a 130% surge in cancellations coupled with an 80% increase in new customers led to a 5% spike in churn. Subsequently, a 48% rise in cancellations and a mere 8% increase in new customers in 2021 resulted in a substantial 10% jump in churn.

**Product 2:** In 2019 a dramatic 190% increase in cancelled customers caused a significant 10% spike in churn. Despite subsequent significant increases in both new customers and cancelled customers (over 100% each), the comparable growth rates led to a relatively stable churn rate.

**Overall, the data demonstrates that cancellations are a primary driver of churn for both products. Addressing this issue is essential for enhancing customer retention.**  

**Key Insight:** The spike in cancellations observed in 2019 for both products suggests a potential underlying issue that was subsequently addressed. It is crucial to monitor for similar trends in the future and take proactive measures to prevent a recurrence


In [0]:
%sql
select year(date_time) as year, month(date_time) as month,reason, count(case_id) as cases from customer_cases left join customer_product on customer_cases.customer_id=customer_product.customer_id
where product = 'prd_2'
group by year(date_time),month(date_time),reason
order by year,month,reason;

year,month,reason,cases
2017,1,signup,244
2017,1,support,26
2017,2,signup,224
2017,2,support,54
2017,3,signup,263
2017,3,support,89
2017,4,signup,249
2017,4,support,122
2017,5,signup,250
2017,5,support,182


In [0]:
import pandas as pd
import plotly.express as px

df_cases= _sqldf.toPandas()
df_filtered = df_cases[df_cases['reason'] == 'signup']
fig = px.line(df_filtered, x='month', y='cases', color='year',
              title='Number of Signup Cases for Monthly Subscriptions')
fig.show()

Monthly model signups exhibit a consistent decline towards the end of each year.

In [0]:
df_filtered = df_cases[df_cases['reason'] == 'support']
fig = px.line(df_filtered, x='month', y='cases', color='year',
              title='Number of Support Cases for Monthly Subscriptions')
fig.show()

While the overall trend is for an increase in cases as the customer base grows, the significant spike in cases observed in 2021 could be a contributing factor to the rise in churn rate.

In [0]:
%sql
select year(date_time) as year, month(date_time) as month,reason, count(case_id) as cases from customer_cases left join customer_product on customer_cases.customer_id=customer_product.customer_id
where product = 'prd_1'
group by year(date_time),month(date_time),reason
order by year,month,reason;

year,month,reason,cases
2017,1,signup,1062
2017,1,support,39
2017,2,signup,959
2017,2,support,113
2017,3,signup,1104
2017,3,support,241
2017,4,signup,933
2017,4,support,322
2017,5,signup,1049
2017,5,support,413


In [0]:
df_cases= _sqldf.toPandas()
df_filtered = df_cases[df_cases['reason'] == 'signup']
fig = px.line(df_filtered, x='month', y='cases', color='year',
              title='Number of Signup Cases for Annual Subscriptions')
fig.show()

While annual subscription models consistently outperform monthly models in terms of signups, both exhibit a common trend of declining towards the end of each year.
However, a notable difference emerged in 2019, where annual subscriptions experienced a significant surge compared to 2020 and 2021. This surge likely contributed to the relatively stable churn rate in 2019, despite a spike in cancelled customers. And an increased churn rate in 2021

In [0]:
df_filtered = df_cases[df_cases['reason'] == 'support']
fig = px.line(df_filtered, x='month', y='cases', color='year',
              title='Number of Support Cases for Annual Subscriptions')
fig.show()

Unlike the monthly subscription model, the annual subscription model has experienced a steady increase in support cases towards the end of each year. This trend was particularly pronounced in 2019, as evidenced by the steepest slope in the line chart for that year.